In [1]:
# Analysis of Retirement with Mutual Funds
# Analysis period from 1999 Jan to 2020 May

#Importing libraries
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [2]:
# install xlrd to read spreadsheets
!pip install xlrd
import xlrd
print('xlrd installed!')

xlrd installed!


In [6]:
# reading excel file into data frame
df_data = pd.read_excel('nav_history.xlsx',
                       sheet_name='Sheet1')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [7]:
# checking data frame
df_data.head()

,Date,Liquid Fund – NAV,Equity Fund – NAV
0,"Jan 01, 1999",11.41,9.29
1,"Feb 01, 1999",11.52,10.23
2,"Mar 01, 1999",11.62,11.64
3,"Apr 05, 1999",11.74,12.62
4,"May 03, 1999",11.84,11.14


In [8]:
# view df details
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               257 non-null    object 
 1   Liquid Fund – NAV  257 non-null    float64
 2   Equity Fund – NAV  257 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.1+ KB


In [ ]:
# Initial monthly income at 1999 Jan 1 is set to ₹12,000. Reason: at 7% inflation, ₹12,000 ~ ₹50,000 as on 2020
# Initial investment is set so that total withdrawal per year is 4% of capital. in this case ₹36 Lakhs
# Asset allocation is set to 60% Equity and 40% Debt